In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

In [2]:
# get the 4th root for embedding: https://developers.googleblog.com/2017/11/introducing-tensorflow-feature-columns.html

## The Preprocessed Data

In [3]:
# the combined forex and economic news data
df = pd.read_csv('fx_with_news.csv', header=[0,1], index_col=0)

In [4]:
df

c_eur                                           \
                         mean       std bar_len bar_spearman bar_log_r   
ctime                                                                    
2016-01-03 22:00:00  1.087467  0.000015    28.0    -0.737459 -0.000037   
2016-01-03 22:05:00  1.087362  0.000049   155.0    -0.181051 -0.000230   
2016-01-03 22:10:00  1.086980  0.000105   138.0    -0.460522 -0.000248   
2016-01-03 22:15:00  1.086938  0.000043   125.0     0.282268 -0.000028   
2016-01-03 22:20:00  1.087057  0.000148   115.0    -0.043127 -0.000120   
...                       ...       ...     ...          ...       ...   
2019-12-31 21:35:00  1.121460  0.000037    62.0    -0.800736 -0.000080   
2019-12-31 21:40:00  1.121460  0.000028    41.0    -0.141550 -0.000009   
2019-12-31 21:45:00  1.121388  0.000046   108.0     0.612689 -0.000018   
2019-12-31 21:50:00  1.121327  0.000046    97.0    -0.912678 -0.000143   
2019-12-31 21:55:00  1.121247  0.000131    91.0     0.075062  0.000294   

                                                                               \
                      first_r     max_r     min_r    last_r bar_quantile_25_r   
ctime                                                                           
2016-01-03 22:00:00  0.000030  0.000030 -0.000016 -0.000007         -0.000009   
2016-01-03 22:05:00  0.000081  0.000081 -0.000149 -0.000149         -0.000029   
2016-01-03 22:10:00  0.000203  0.000203 -0.000147 -0.000046         -0.000064   
2016-01-03 22:15:00 -0.000007  0.000057 -0.000108 -0.000035         -0.000007   
2016-01-03 22:20:00 -0.000144  0.000224 -0.000264 -0.000264         -0.000126   
...                       ...       ...       ...       ...               ...   
2019-12-31 21:35:00  0.000062  0.000062 -0.000036 -0.000018         -0.000027   
2019-12-31 21:40:00 -0.000027  0.000035 -0.000054 -0.000036         -0.000018   
2019-12-31 21:45:00  0.000029  0.000047 -0.000096  0.000011         -0.000025   
2019-12-31 21:50:00  0.000056  0.000092 -0.000122 -0.000086         -0.000015   
2019-12-31 21:55:00 -0.000033  0.000261 -0.000123  0.000261         -0.000078   

                     ...  \
                     ...   
ctime                ...   
2016-01-03 22:00:00  ...   
2016-01-03 22:05:00  ...   
2016-01-03 22:10:00  ...   
2016-01-03 22:15:00  ...   
2016-01-03 22:20:00  ...   
...                  ...   
2019-12-31 21:35:00  ...   
2019-12-31 21:40:00  ...   
2019-12-31 21:45:00  ...   
2019-12-31 21:50:00  ...   
2019-12-31 21:55:00  ...   

                                                             after_counter_ohe  \
                    _united states nondefense capital goods orders ex aircraft   
ctime                                                                            
2016-01-03 22:00:00                                           0.000000           
2016-01-03 22:05:00                                           0.000000           
2016-01-03 22:10:00                                           0.000000           
2016-01-03 22:15:00                                           0.000000           
2016-01-03 22:20:00                                           0.000000           
...                                                                ...           
2019-12-31 21:35:00                                           0.429514           
2019-12-31 21:40:00                                           0.429167           
2019-12-31 21:45:00                                           0.428819           
2019-12-31 21:50:00                                           0.428472           
2019-12-31 21:55:00                                           0.428125           

                                                     \
                    _united states nonfarm payrolls   
ctime                                                 
2016-01-03 22:00:00                             0.0   
2016-01-03 22:05:00                             0.0   
2016-01-03 22:10:00                

In [5]:
# list of currencies
c_list = ['c_eur', 'c_gbp', 'c_jpy']

In [6]:
# make new feature for more stable forecast
for cur in c_list:    
    # logreturn of the means
    df[(cur,'mean_log_r')] = np.log(1.0 + df[(cur,'mean')].pct_change())
    df[(cur,'mean_log_r')] = df[(cur,'mean_log_r')].fillna(0.0)

In [7]:
# column hierarchy
for top in list(df.columns.get_level_values(0).unique()):
    print('----------------------------------')
    print('TOP COLUMN: ', top)
    print('SUBCOLUMNS: ', len(df[top].columns))
    print(list(df[top].columns))


----------------------------------
TOP COLUMN:  c_eur
SUBCOLUMNS:  12
['mean', 'std', 'bar_len', 'bar_spearman', 'bar_log_r', 'first_r', 'max_r', 'min_r', 'last_r', 'bar_quantile_25_r', 'bar_quantile_75_r', 'mean_log_r']
----------------------------------
TOP COLUMN:  c_gbp
SUBCOLUMNS:  12
['mean', 'std', 'bar_len', 'bar_spearman', 'bar_log_r', 'first_r', 'max_r', 'min_r', 'last_r', 'bar_quantile_25_r', 'bar_quantile_75_r', 'mean_log_r']
----------------------------------
TOP COLUMN:  c_jpy
SUBCOLUMNS:  12
['mean', 'std', 'bar_len', 'bar_spearman', 'bar_log_r', 'first_r', 'max_r', 'min_r', 'last_r', 'bar_quantile_25_r', 'bar_quantile_75_r', 'mean_log_r']
----------------------------------
TOP COLUMN:  month
SUBCOLUMNS:  12
['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12']
----------------------------------
TOP COLUMN:  dow
SUBCOLUMNS:  6
['_0', '_1', '_2', '_3', '_4', '_6']
----------------------------------
TOP COLUMN:  hour
SUBCOLUMNS:  24
['_0', '_1', '_2',

In [8]:
top_level = list(df.columns.get_level_values(0).unique())
top_level

['c_eur',
 'c_gbp',
 'c_jpy',
 'month',
 'dow',
 'hour',
 'event_cur',
 'event_exist',
 'actual_ohe',
 'surprise_ohe',
 'change_ohe',
 'after_counter_ohe']

## Processing for Model Input

In [9]:
# we will use 2016-2018 as training data, and 2019 as validation and test data
# problem: we have events that occure only in 2019
# we can't normalize/standardize/scale them based on 2016-2018 data
# we could move them to the '_(currency) event' columns, 
# but there is only a few, so here I will delete them

# have we all event type in 2016-2018 yers?
df['event_exist'].loc['2016':'2019'].any().all()

False

In [10]:
# find events occuring only in 2019
event_before_19 = df['event_exist'].loc['2016':'2019'].any()
event_only_19 = list(event_before_19[event_before_19==False].index)
event_only_19

['_european monetary union markit pmi composite',
 '_germany zew survey  economic sentiment',
 '_japan gross domestic product qoq',
 '_united states consumer price index ex food  energy mom',
 '_united states initial jobless claims 4week average',
 '_united states nondefense capital goods orders ex aircraft']

In [11]:
# drop events only occuring in 2019
df.drop(event_only_19, axis=1, level=1, inplace=True)

In [12]:
# have we all event type in 2016-2018 yers?
df['event_exist'].loc['2016':'2019'].any().all()
#ok

True

In [13]:
# rescaling output isn't always important, 
# but if the model has different output features it helps to weight their effect on the loss function
# or we can use multiple loss functions and manually set the weight
df_label = df.xs('mean', axis=1, level=1, drop_level=False)
df_label

,c_eur,c_gbp,c_jpy
,mean,mean,mean
ctime,,,
2016-01-03 22:00:00,1.087467,1.473700,0.831731
2016-01-03 22:05:00,1.087362,1.474204,0.831720
2016-01-03 22:10:00,1.086980,1.474567,0.831713
2016-01-03 22:15:00,1.086938,1.474208,0.831647
2016-01-03 22:20:00,1.087057,1.474157,0.831599
...,...,...,...
2019-12-31 21:35:00,1.121460,1.324929,0.920382
2019-12-31 21:40:00,1.121460,1.325454,0.920438


In [14]:
# we use only train and validation
# Standardizing and normalizing shifts the dataset, where +- sign can be important. Later the model will adjust this by the biases,
# but the information is lost. It could improve the performance if we make a new feature for + and - numbers, but here I don't want to make more features

In [15]:
# spearman doesn't need standardization
c_single_features = ['mean', 'std', 'bar_len']
feature_scalers = dict()

In [16]:
# standardize features based on data up to 2019-01-01 00:00
for cur in c_list:
    for feature in c_single_features:
        column_mean = df[cur, feature].loc['2016':'2019'].mean()
        column_std = df[cur, feature].loc['2016':'2019'].std()
        feature_scalers.update({(cur, feature) : [column_mean, column_std]})
        df.loc[:,(cur, feature)] = (df[cur, feature] - column_mean)/column_std

In [17]:
# !!! later we can use them to rescale if needed
feature_scalers # column_mean, column_std

{('c_eur', 'bar_len'): [412.9294628759021, 381.7685744752972],
 ('c_eur', 'mean'): [1.139248301570798, 0.0499815489615703],
 ('c_eur', 'std'): [0.00010543836669119138, 9.077071115943524e-05],
 ('c_gbp', 'bar_len'): [387.5186401063365, 303.3104183503982],
 ('c_gbp', 'mean'): [1.326336873217585, 0.06648514955410888],
 ('c_gbp', 'std'): [0.000146642848991849, 0.00015890007722327068],
 ('c_jpy', 'bar_len'): [354.61210380295637, 299.81779856625644],
 ('c_jpy', 'mean'): [0.9066954709408044, 0.03345136241766339],
 ('c_jpy', 'std'): [9.800579913539747e-05, 9.589094629696171e-05]}

In [18]:
c_related_features = ['mean_log_r','bar_log_r','first_r','max_r','min_r','last_r','bar_quantile_25_r','bar_quantile_75_r']
# these are related feature, we can think of them as one feature with subfeatures
# they are on similar scale 
# and the pairs on the other side of the mean (like max_r and min_r) are almost symmetric
# to preserve the signs we don't subtract the mean, only divide by the std
# this way we preserve their relative size
# *10000000 is only for easier comparison
print(df['c_eur'][c_related_features].abs().mean()*10000000)
print(df['c_eur'][c_related_features].mean()*10000000)

mean_log_r           1435.641469
bar_log_r            1756.307094
first_r              1079.158746
max_r                1680.077984
min_r                1686.788765
last_r                960.660734
bar_quantile_25_r     647.482120
bar_quantile_75_r     647.172621
dtype: float64
mean_log_r              1.023680
bar_log_r               8.380970
first_r                -0.840415
max_r                1680.077984
min_r               -1686.788765
last_r                  7.540555
bar_quantile_25_r    -647.041955
bar_quantile_75_r     646.659518
dtype: float64


In [19]:
related_std = df[cur, 'bar_log_r'].loc['2016':'2019'].std()

for cur in c_list:
    for feature in c_related_features:
        df.loc[:,(cur, feature)] = (df[cur, feature])/related_std

In [20]:
# !!! needed later to rescale outputs of log_r features
related_std

0.0003561751218763309

In [21]:
# ok, these features remained on similar scale, but the other features as well
print(df['c_eur'][c_related_features].abs().mean())
print(df['c_eur'][c_related_features].mean())

mean_log_r           0.403072
bar_log_r            0.493102
first_r              0.302985
max_r                0.471700
min_r                0.473584
last_r               0.269716
bar_quantile_25_r    0.181788
bar_quantile_75_r    0.181701
dtype: float64
mean_log_r           0.000287
bar_log_r            0.002353
first_r             -0.000236
max_r                0.471700
min_r               -0.473584
last_r               0.002117
bar_quantile_25_r   -0.181664
bar_quantile_75_r    0.181557
dtype: float64


In [22]:
# 'event_cur' is on similar scale, we don't touch (normalization would be better than standardization)
df['event_cur'].max()

_EUR    2.000000
_GBP    2.828427
_JPY    1.732051
_USD    3.000000
dtype: float64

In [23]:
# should consider every event type alone, but there are 76 event types, so just normalize
df['actual_ohe'] = (df['actual_ohe'] - df['actual_ohe'].loc['2016':'2019'].min())/(df['actual_ohe'].loc['2016':'2019'].max() - df['actual_ohe'].loc['2016':'2019'].min())
# where max-min == 0 we get NaN, fill with 0
df['actual_ohe'] = df['actual_ohe'].fillna(0.0)

In [24]:
# dividing with the max of absolute values keeps the signs
df['surprise_ohe'] = df['surprise_ohe']/df['surprise_ohe'].loc['2016':'2019'].abs().max()
df['surprise_ohe'] = df['surprise_ohe'].fillna(0.0)

df['change_ohe'] = df['change_ohe']/df['change_ohe'].loc['2016':'2019'].abs().max()
df['change_ohe'] = df['change_ohe'].fillna(0.0)

df['after_counter_ohe'] = df['after_counter_ohe']/df['after_counter_ohe'].loc['2016':'2019'].abs().max()
df['after_counter_ohe'] = df['after_counter_ohe'].fillna(0.0)

In [25]:
# df_label = df_label.astype('float16')
# df = df.astype('float16')
df_valid = df.loc['2019':]
df = df.loc[:'2019']
df_label_valid = df_label.loc['2019':]
df_label = df_label.loc[:'2019']

In [26]:
print(df.std().max())
print(df.std().mean())

1.1175694609368847
0.23388258061315387


## The Datasets

In [27]:
train_size=144
shift=1
batch_size=64

In [28]:
# label data begins from the end of the first train data

def ds_input(df, concat_tops, feature_to_last):
    nx = df.to_numpy()
    if (isinstance(df.columns, pd.core.index.MultiIndex) and concat_tops==False):
        top_level_nb = len(df.columns.get_level_values(0).unique())
        sub_level_nb = len(df.columns.get_level_values(1).unique())
        nx = nx.reshape(-1,top_level_nb,sub_level_nb)
        if feature_to_last == True:
            nx = np.moveaxis(nx, [0,1,2], [0,2,1])
        nx = np.squeeze(nx)
    return nx

def make_ds(batch_size, time_steps, shift, skip_steps, df, concat_tops=False, feature_to_last=True):
    nx = ds_input(df, concat_tops, feature_to_last)
    ds = tf.data.Dataset.from_tensor_slices(nx[skip_steps:])
    ds = ds.window(time_steps, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(time_steps))
    return ds.batch(batch_size, drop_remainder=True)

In [29]:
label_dataset_args = {'batch_size':batch_size,
                      'time_steps':train_size,
                      'shift':shift, 
                      'skip_steps':1}

ds_label = make_ds(**label_dataset_args, df=df_label)
ds_label_valid = make_ds(**label_dataset_args, df=df_label_valid)

In [34]:
input_dataset_args = {'batch_size':batch_size,
                      'time_steps':train_size,
                      'shift':shift, 
                      'skip_steps':0}

ds_full = make_ds(**input_dataset_args, df=df, concat_tops=True)
ds_full_valid = make_ds(**input_dataset_args, df=df_valid, concat_tops=True)

ds_curbars = make_ds(**input_dataset_args, df=df[['c_eur', 'c_gbp', 'c_jpy']], concat_tops=True)
ds_curbars_valid = make_ds(**input_dataset_args, df=df_valid[['c_eur', 'c_gbp', 'c_jpy']], concat_tops=True)

ds_event_exist = make_ds(**input_dataset_args, df=df['event_exist'])
ds_event_exist_valid = make_ds(**input_dataset_args, df=df_valid['event_exist'])

In [35]:
future_dataset_args = {'batch_size':batch_size,
                      'time_steps':train_size,
                      'shift':shift, 
                      'skip_steps':1}

ds_future = make_ds(**future_dataset_args, df=df[['event_exist', 'month', 'dow', 'hour', 'event_cur']], concat_tops=True)
ds_future_valid = make_ds(**future_dataset_args, df=df_valid[['event_exist', 'month', 'dow', 'hour', 'event_cur']], concat_tops=True)

In [36]:
print('ds_label ',ds_label)

print('ds_full ', ds_full)
print('ds_curbars ', ds_curbars)
print('ds_event_exist ', ds_event_exist)

print('ds_future ', ds_future)

ds_label  <BatchDataset shapes: (64, None, 3), types: tf.float64>
ds_full  <BatchDataset shapes: (64, None, 432), types: tf.float64>
ds_curbars  <BatchDataset shapes: (64, None, 36), types: tf.float64>
ds_event_exist  <BatchDataset shapes: (64, None, 70), types: tf.float64>
ds_future  <BatchDataset shapes: (64, None, 116), types: tf.float64>


## Building the Model

In [37]:
class GatedActivationUnit(keras.layers.Layer):
    def __init__(self, activation="tanh", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'activation': self.activation,
        })
        return config 
    
    def call(self, inputs):
        n_filters = inputs.shape[-1] // 2
        linear_output = self.activation(inputs[..., :n_filters])
        gate = keras.activations.sigmoid(inputs[..., n_filters:])
        return self.activation(linear_output) * gate

In [38]:
def wavenet_residual_block(inputs, n_filters, dilation_rate):
    z = keras.layers.Conv1D(2 * n_filters, kernel_size=2, padding="causal",
                            dilation_rate=dilation_rate)(inputs)
    z = GatedActivationUnit()(z)
    z = keras.layers.Conv1D(n_filters, kernel_size=1)(z)
    return keras.layers.Add()([z, inputs]), z

In [39]:
def wavenet_model_setup(n_layers_per_block, n_blocks, n_filters, n_outputs, feature_dim, name):
    # n_layers_per_block = 10 in the paper
    # n_blocks = 3 in the paper
    # n_filters = 128 in the paper
    # n_outputs = 256 in the paper
    
    inputs = keras.layers.Input(shape=[train_size, feature_dim]) # 164 with cross model
    z = keras.layers.Conv1D(n_filters, kernel_size=2, padding="causal")(inputs)
    skip_to_last = []
    for dilation_rate in [2**i for i in range(n_layers_per_block)] * n_blocks:
        z, skip = wavenet_residual_block(z, n_filters, dilation_rate)
        skip_to_last.append(skip)
    z = keras.activations.relu(keras.layers.Add()(skip_to_last))
    z = keras.layers.Conv1D(n_filters, kernel_size=1, activation="relu")(z)
    out = keras.layers.Conv1D(n_outputs, kernel_size=1, activation="relu")(z)
    return keras.models.Model(inputs=[inputs], outputs=[out], name = name)

In [68]:
def model_event_exist_setup(inputs, embed_dim, name=None):
    out = keras.layers.Conv1D(embed_dim, kernel_size=1, strides=1, padding='valid')(inputs)
    return keras.models.Model(inputs=[inputs], outputs=[out])

In [70]:
# inputs
input_event_exist = keras.layers.Input(shape=[train_size, 70], name = 'input_event_exist') 
input_curbars = keras.layers.Input(shape=[train_size, 36], name = 'input_curbars')
input_full = keras.layers.Input(shape=[train_size, 432], name = 'input_full')

input_future = keras.layers.Input(shape=[train_size, 116], name = 'input_future')

## Train Simplest model

In [41]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [45]:
model_wn = wavenet_model_setup(n_layers_per_block=4, n_blocks=2, n_filters=64, n_outputs=3, feature_dim=36, name='mod_wavenet_fl64_fd36')
model_wn.summary()

Model: "mod_wavenet_fl64_fd36"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 144, 36)]    0                                            
__________________________________________________________________________________________________
conv1d_19 (Conv1D)              (None, 144, 64)      4672        input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_20 (Conv1D)              (None, 144, 128)     16512       conv1d_19[0][0]                  
__________________________________________________________________________________________________
gated_activation_unit_8 (GatedA (None, 144, 64)      0           conv1d_20[0][0]                  
______________________________________________________________________________

In [43]:
wave_out = model_wn(input_curbars)

model_no_f = keras.models.Model(inputs=[input_curbars], outputs=[wave_out])

model_no_f.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_curbars (InputLayer)   [(None, 144, 36)]         0         
_________________________________________________________________
mod_wavenet_fl64_fd36 (Model (None, 144, 3)            174403    
Total params: 174,403
Trainable params: 174,403
Non-trainable params: 0
_________________________________________________________________


In [44]:
ds_train = tf.data.Dataset.zip((ds_curbars, ds_label)).shuffle(128, reshuffle_each_iteration=True).prefetch(tf.data.experimental.AUTOTUNE)
ds_valid = tf.data.Dataset.zip((ds_curbars_valid, ds_label_valid)).prefetch(tf.data.experimental.AUTOTUNE)

In [54]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model_no_f.compile(loss=keras.losses.Huber(), optimizer=optimizer, metrics=["mae"])
checkpoint_cb = keras.callbacks.ModelCheckpoint("model_wn_no_f.h5", save_best_only=True)

history = model_no_f.fit(ds_train, epochs=14,
                       validation_data=ds_valid,
                       callbacks=[checkpoint_cb])

Epoch 1/14
3500/3500 [==============================] - 198s 56ms/step - loss: 6.6098e-04 - mae: 0.0130 - val_loss: 5.4135e-05 - val_mae: 0.0068
Epoch 2/14
3500/3500 [==============================] - 187s 53ms/step - loss: 6.8699e-05 - mae: 0.0065 - val_loss: 1.2946e-05 - val_mae: 0.0038
Epoch 3/14
3500/3500 [==============================] - 183s 52ms/step - loss: 4.7091e-05 - mae: 0.0052 - val_loss: 2.9264e-05 - val_mae: 0.0066
Epoch 4/14
3500/3500 [==============================] - 183s 52ms/step - loss: 4.2217e-05 - mae: 0.0048 - val_loss: 5.3478e-06 - val_mae: 0.0027
Epoch 5/14
3500/3500 [==============================] - 183s 52ms/step - loss: 4.1491e-05 - mae: 0.0046 - val_loss: 9.8578e-06 - val_mae: 0.0034
Epoch 6/14
3500/3500 [==============================] - 185s 53ms/step - loss: 3.8956e-05 - mae: 0.0048 - val_loss: 7.2391e-06 - val_mae: 0.0027
Epoch 7/14
3500/3500 [==============================] - 188s 54ms/step - loss: 3.4971e-05 - mae: 0.0047 - val_loss: 7.8174e-06 - v

## Train Simple Model with all input

In [71]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [73]:
model_event_exist = model_event_exist_setup(input_event_exist, 6, name='mod_event_exist')
model_event_exist.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_event_exist (InputLaye [(None, 144, 70)]         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 144, 6)            426       
Total params: 426
Trainable params: 426
Non-trainable params: 0
_________________________________________________________________


In [77]:
model_wn = wavenet_model_setup(n_layers_per_block=4, n_blocks=2, n_filters=64, n_outputs=3, feature_dim=42, name='mod_wavenet_fl64_fd42')

In [78]:
emb_event_exist = model_event_exist(input_event_exist)

input_wn = keras.layers.Concatenate(axis=-1)([input_curbars, emb_event_exist])

wave_out = model_wn(input_wn)

model_no_f_ev_ex = keras.models.Model(inputs=[input_curbars, input_event_exist], outputs=[wave_out])

model_no_f_ev_ex.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_event_exist (InputLayer)  [(None, 144, 70)]    0                                            
__________________________________________________________________________________________________
input_curbars (InputLayer)      [(None, 144, 36)]    0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 144, 6)       426         input_event_exist[0][0]          
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 144, 42)      0           input_curbars[0][0]              
                                                                 model_1[3][0]              

In [79]:
ds_train = tf.data.Dataset.zip(((ds_curbars, ds_event_exist), ds_label)).shuffle(128, reshuffle_each_iteration=True).prefetch(tf.data.experimental.AUTOTUNE)
ds_valid = tf.data.Dataset.zip(((ds_curbars_valid, ds_event_exist_valid), ds_label_valid)).prefetch(tf.data.experimental.AUTOTUNE)

In [81]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model_no_f_ev_ex.compile(loss=keras.losses.Huber(), optimizer=optimizer, metrics=["mae"])
checkpoint_cb = keras.callbacks.ModelCheckpoint("model_nofut_fullin.h5", save_best_only=True)

history = model_no_f_ev_ex.fit(ds_train, epochs=14,
                       validation_data=ds_valid,
                       callbacks=[checkpoint_cb])

Epoch 1/14
3500/3500 [==============================] - 297s 85ms/step - loss: 8.8564e-04 - mae: 0.0133 - val_loss: 7.0183e-05 - val_mae: 0.0097
Epoch 2/14
3500/3500 [==============================] - 291s 83ms/step - loss: 7.2995e-05 - mae: 0.0061 - val_loss: 7.5863e-06 - val_mae: 0.0029
Epoch 3/14
3500/3500 [==============================] - 287s 82ms/step - loss: 7.4844e-05 - mae: 0.0061 - val_loss: 1.2169e-05 - val_mae: 0.0034
Epoch 4/14
3500/3500 [==============================] - 290s 83ms/step - loss: 4.7924e-05 - mae: 0.0046 - val_loss: 1.5435e-05 - val_mae: 0.0035
Epoch 5/14
3500/3500 [==============================] - 289s 83ms/step - loss: 3.3789e-05 - mae: 0.0046 - val_loss: 9.7458e-06 - val_mae: 0.0029
Epoch 6/14
3500/3500 [==============================] - 287s 82ms/step - loss: 3.3019e-05 - mae: 0.0047 - val_loss: 9.6791e-06 - val_mae: 0.0032
Epoch 7/14
3500/3500 [==============================] - 281s 80ms/step - loss: 2.0366e-05 - mae: 0.0037 - val_loss: 9.3054e-06 - v

## Train with price and return loss

In [46]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
wave_out = model_wn(input_full)

'''
input_f = keras.layers.Conv1D(32, kernel_size=1, strides=1, padding='valid', activation='relu')(input_future)

ftr = keras.layers.Concatenate(axis=-1)([out_timed, input_f])
ftr = keras.layers.Conv1D(24, kernel_size=1, strides=1, padding='valid', activation='relu')(ftr)
ftr = keras.layers.BatchNormalization()(ftr)
out_f = keras.layers.Conv1D(3, kernel_size=1, strides=1, padding='valid', activation=None)(ftr)
'''

model_no_f = keras.models.Model(inputs=[input_full], outputs=[wave_out])

model_no_f.summary()

In [65]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model_sum.compile(loss=weighted_mae_loss_setup(5), optimizer=optimizer, metrics=["mae", mae_mean_first])
checkpoint_cb = keras.callbacks.ModelCheckpoint("model_wn_simple.h5", save_best_only=True)

history = model_sum.fit(ds_train, epochs=14,
                       validation_data=ds_valid,
                       callbacks=[checkpoint_cb])

Epoch 1/14
   3500/Unknown - 209s 60ms/step - loss: 0.3232 - mae: 0.5002 - mae_mean_first: 0.4993

RuntimeError: Unable to create link (name already exists)

## Evaluate Model

In [39]:
model_direct = keras.models.load_model("model_wn_no_f.h5", custom_objects={'GatedActivationUnit' : GatedActivationUnit})

In [ ]:
model_direct.evaluate(ds_valid)

In [40]:
pred = model_direct.predict(ds_train.take(3))

In [47]:
next(iter(ds_label_valid))

<tf.Tensor: shape=(64, 144, 3), dtype=float64, numpy=
array([[[1.14660213, 1.27502506, 0.91176042],
        [1.14665078, 1.27506083, 0.91166034],
        [1.14665252, 1.2750439 , 0.91146625],
        ...,
        [1.14387721, 1.26764373, 0.91705545],
        [1.14373084, 1.26736633, 0.91678977],
        [1.14366858, 1.26727479, 0.9167313 ]],

       [[1.14665078, 1.27506083, 0.91166034],
        [1.14665252, 1.2750439 , 0.91146625],
        [1.14670331, 1.27497803, 0.91151996],
        ...,
        [1.14373084, 1.26736633, 0.91678977],
        [1.14366858, 1.26727479, 0.9167313 ],
        [1.14336997, 1.2670677 , 0.91643917]],

       [[1.14665252, 1.2750439 , 0.91146625],
        [1.14670331, 1.27497803, 0.91151996],
        [1.14674477, 1.27492272, 0.91152943],
        ...,
        [1.14366858, 1.26727479, 0.9167313 ],
        [1.14336997, 1.2670677 , 0.91643917],
        [1.14283442, 1.2668242 , 0.91627286]],

       ...,

       [[1.14452395, 1.27308128, 0.91436697],
        [1.144

In [42]:
pred[15000]

IndexError: index 15000 is out of bounds for axis 0 with size 192

## OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO

In [50]:
df_label[10000:10020]

,c_eur,c_gbp,c_jpy
,mean_log_r,mean_log_r,mean_log_r
ctime,,,
2016-02-19 15:20:00,0.356657,0.460169,-0.382856
2016-02-19 15:25:00,1.451761,-0.149109,1.167526
2016-02-19 15:30:00,-0.762259,1.091631,-0.608134
2016-02-19 15:35:00,-1.157844,0.840024,-0.814383
2016-02-19 15:40:00,0.138421,0.492306,0.455284
2016-02-19 15:45:00,0.316235,-0.310320,0.520709
2016-02-19 15:50:00,0.403279,-0.080827,-0.010239
2016-02-19 15:55:00,-0.433683,-1.497703,0.609927


## Train Simple with decoupled features

In [ ]:
ds_train = tf.data.Dataset.zip(((ds_bars_simple, ds_event_ohe, ds_time_cur, ds_event_exist, ds_time_curr_future, ds_event_future), 
                                 ds_label_means)).prefetch(tf.data.experimental.AUTOTUNE)
ds_valid = tf.data.Dataset.zip(((ds_bars_simple_valid, ds_event_ohe_valid, ds_time_cur_valid, ds_event_exist_valid, ds_time_curr_future_valid, ds_event_future_valid), 
                                 ds_label_means_valid)).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# future and past embeds are different
model_time_cur_f = model_time_cur_setup(input_f_time_cur, 6, name='model_time_cur_f')
model_event_exist_f = model_event_exist_setup(input_f_event_exist, 10, name='model_event_exist_f')

In [ ]:
ohe = model_event_ohe(input_ohe)
time_cur = model_time_cur(input_time_cur)
event_ex = model_event_exist(input_event_exist)
inputs = keras.layers.Concatenate(axis=-1)([input_simple_c, ohe, time_cur, event_ex])
wave = model_wn(inputs)
out_head = model_time_transform(wave)

time_cur_f = model_time_cur_f(input_f_time_cur)
event_ex_f = model_event_exist_f(input_f_event_exist)

input_f = keras.layers.Concatenate(axis=-1)([out_head, time_cur_f, event_ex_f])
out_f = keras.layers.Conv1D(3, kernel_size=1, strides=1, padding='valid')(input_f)

model_sum = keras.models.Model(inputs=[input_simple_c, input_ohe, input_time_cur, input_event_exist, input_f_time_cur, input_f_event_exist], outputs=[out_f])

model_sum.summary()

In [ ]:
optimizer = keras.optimizers.Adam(lr=0.001)
model_sum.compile(loss=keras.losses.Huber(), optimizer=optimizer, metrics=["mae", mean_metric(last_index=None)])
checkpoint_cb = keras.callbacks.ModelCheckpoint("model_wn_decoup.h5", save_best_only=True)

history = model_sum.fit(ds_train, epochs=14,
                       validation_data=ds_valid,
                       callbacks=[checkpoint_cb])

## Redefine Head Model

In [ ]:
def head_model2_setup():
    inputs = keras.layers.Input(shape=[train_size, 3])
    z = keras.layers.BatchNormalization()(inputs)
    z = keras.layers.Conv1D(8, kernel_size=5, strides=3, padding='valid', activation='relu')(z)
    z = keras.layers.BatchNormalization()(z)
    z = keras.layers.Conv1D(16, kernel_size=3, strides=2, padding='valid', activation='relu')(z)
    z = keras.layers.BatchNormalization()(z)
    z = keras.layers.Conv1D(32, kernel_size=3, strides=2, padding='valid', activation='relu')(z)
    out = keras.layers.BatchNormalization()(z)
    return keras.models.Model(inputs=[inputs], outputs=[out], name = 'model_head2')

model_head2 = head_model2_setup()
model_head2.summary()

In [ ]:
# bottleneck?
model_event_ohe = model_event_ohe_setup(input_ohe, 64)
model_event_ohe.summary()

In [ ]:
# +48 features
model_wn = wavenet_model_setup(n_layers_per_block=3, n_blocks=2, n_filters=64, n_outputs=3, feature_dim=113, name='model_wn_fl64_fd81')
# model_wn_out = wavenet_model_setup(n_layers_per_block=3, n_blocks=1, n_filters=64, n_outputs=3, feature_dim=48, name='model_wn_out_fl64_fd_48')

In [ ]:
target_size=71

In [ ]:
ds_label = make_ds(batch_size=batch_size,
                   time_steps=target_size, 
                   shift=shift, 
                   skip_steps=train_size, 
                   df=df_label)

ds_label_valid = make_ds(batch_size=batch_size,
                   time_steps=target_size, 
                   shift=shift, 
                   skip_steps=train_size, 
                   df=df_label_valid)
ds_event_future = make_ds(  batch_size=batch_size,
                            time_steps=target_size, 
                            shift=shift, 
                            skip_steps=train_size, 
                            df=df['event_exist'])
ds_time_curr_future = make_ds(  batch_size=batch_size,
                            time_steps=target_size, 
                            shift=shift, 
                            skip_steps=train_size, 
                            df=df[['month', 'dow', 'hour', 'event_cur']], concat_tops=True)

ds_event_future_valid = make_ds(  batch_size=batch_size,
                            time_steps=target_size, 
                            shift=shift, 
                            skip_steps=train_size, 
                            df=df_valid['event_exist'])
ds_time_curr_future_valid = make_ds(  batch_size=batch_size,
                            time_steps=target_size, 
                            shift=shift, 
                            skip_steps=train_size, 
                            df=df_valid[['month', 'dow', 'hour', 'event_cur']], concat_tops=True)

input_f_time_cur = keras.layers.Input(shape=[target_size, 46], name = 'input_f_time_cur')
input_f_event_exist = keras.layers.Input(shape=[target_size, 70], name = 'input_f_event_exist')

In [ ]:
ohe = model_event_ohe(input_ohe)
time_cur = model_time_cur(input_time_cur)
event_ex = model_event_exist(input_event_exist)
inputs = keras.layers.Concatenate(axis=-1)([input_simple_c, ohe, time_cur, event_ex])
wave = model_wn(inputs)
out_head = model_head2(wave)

time_cur_f = model_time_cur(input_f_time_cur)
event_ex_f = model_event_exist(input_f_event_exist)

input_f = keras.layers.Concatenate(axis=-1)([out_head, time_cur_f, event_ex_f])
input_f = keras.layers.Conv1D(48, kernel_size=3, strides=1, padding='causal', activation='relu')(input_f)
out_f = keras.layers.Conv1D(3, kernel_size=1, strides=1, padding='valid')(input_f)

model_sum = keras.models.Model(inputs=[input_simple_c, input_ohe, input_time_cur, input_event_exist, input_f_time_cur, input_f_event_exist], outputs=[out_f])

model_sum.summary()

In [ ]:
# test if model converge
'''
ds_worker_test = tf.data.Dataset.zip(((ds_bars_simple, ds_event_ohe, ds_time_cur, ds_event_exist, ds_time_curr_future, ds_event_future), 
                                 ds_label)).shuffle(64).prefetch(tf.data.experimental.AUTOTUNE).take(64)
optimizer = keras.optimizers.Adam(lr=0.001)
model_sum.compile(loss=keras.losses.Huber(), optimizer=optimizer, metrics=["mae"])
history = model_sum.fit(ds_worker_test, epochs=3)
'''

In [ ]:
ds_train = tf.data.Dataset.zip(((ds_bars_simple, ds_event_ohe, ds_time_cur, ds_event_exist, ds_time_curr_future, ds_event_future), 
                                 ds_label)).prefetch(tf.data.experimental.AUTOTUNE)
ds_valid = tf.data.Dataset.zip(((ds_bars_simple_valid, ds_event_ohe_valid, ds_time_cur_valid, ds_event_exist_valid, ds_time_curr_future_valid, ds_event_future_valid), 
                                 ds_label_valid)).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
optimizer = keras.optimizers.Adam(lr=0.001)
model_sum.compile(loss=keras.losses.Huber(), optimizer=optimizer, metrics=["mae"])
checkpoint_cb = keras.callbacks.ModelCheckpoint("model_wn_64_head_2.h5", save_best_only=True)

history = model_sum.fit(ds_train, epochs=20,
                       validation_data=ds_valid,
                       callbacks=[checkpoint_cb])

# Train Cross Feature model

In [ ]:
# 
z1 = keras.layers.Conv2D(2*3, kernel_size=1, padding="valid")(input_c1)
z1 = keras.layers.Reshape([train_size,66])(z1)

z2 = keras.layers.Conv2D(2*11, kernel_size=1, padding="valid")(input_c2)
z2 = keras.layers.Reshape([train_size,66])(z2)

out = keras.layers.Concatenate(axis=-1)([z1, z2])
model_cur_cross_in = keras.models.Model(inputs=[input_c1, input_c2], outputs=[out])
model_cur_cross_in._name = 'model_cur_cross_in'

model_cur_cross_in.summary()

In [ ]:
model_wn = wavenet_model(n_layers_per_block=3, n_blocks=2, n_filters=256, n_outputs=3, feature_dim=164)
# (fast test) #  model_wn = wavenet_model(n_layers_per_block=2, n_blocks=1, n_filters=1, n_outputs=3)
model_wn._name = 'model_wavenet'

#keras.utils.plot_model(model, show_shapes=True)
model_wn.summary()

In [ ]:
ohe = model_event_ohe(input_ohe)
time_cur = model_time_cur(input_time_cur)
event_ex = model_event_exist(input_event_exist)
cur_cross = model_cur_cross_in([input_c1, input_c2])
inputs = keras.layers.Concatenate(axis=-1)([cur_cross, ohe, time_cur, event_ex])
wave = model_wn(inputs)
out_head = model_head(wave)

time_cur_f = model_time_cur(input_f_time_cur)
event_ex_f = model_event_exist(input_f_event_exist)

input_f = keras.layers.Concatenate(axis=-1)([out_head, time_cur_f, event_ex_f])
out_f = keras.layers.Conv1D(3, kernel_size=1, strides=1, padding='valid')(input_f)

model_sum = keras.models.Model(inputs=[input_c1, input_c2, input_ohe, input_time_cur, input_event_exist, input_f_time_cur, input_f_event_exist], outputs=[out_f])

model_sum.summary()

In [ ]:
out = keras.layers.Conv1D(64, kernel_size=1, strides=1, padding='valid')(input_ohe)
model_event_ohe = keras.models.Model(inputs=[input_ohe], outputs=[out])
model_event_ohe._name = 'model_event_ohe'

model_event_ohe.summary()

In [ ]:
label_transform = dict()
# standardize features based on data up to 2019-01-01 00:00
for cur in c_list:
    mean_to_19 = df_label[cur, 'mean'].loc['2016':'2019'].mean()
    std_to_19 = df_label[cur, 'mean'].loc['2016':'2019'].std()
    label_transform.update({(cur, 'mean') : {'column_mean' : mean_to_19, 'column_std' : std_to_19}})
    #df_label.loc[:,(cur, 'mean')] = (df_label.loc[:,(cur, 'mean')] - mean_to_19)/std_to_19